In [1]:
import autogen
import ollama
from typing import List, Dict
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader
import os
import glob
import pandas as pd
from docx import Document
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import uuid

# 導入自定義的 JSONVectorDB
from vector_db import JSONVectorDB
class CustomRAGAgentSystem:
    def __init__(self, reset_db=False, db_path="./custom_json_rag_db"):
        # Ollama配置
        self.base_url = "http://localhost:11434/v1"
        self.embedding_model = "tsd_4500datas_summary20250606_epoch11_f32:latest"
        self.llm_model = "qwen3:14b"
        
        # 初始化自定義JSON資料庫
        self.db_path = db_path
        if reset_db and os.path.exists(db_path):
            import shutil
            shutil.rmtree(db_path)
        
        # 創建自定義資料庫實例
        self.collection = JSONVectorDB(db_path)
        
        # 如果需要重置
        if reset_db:
            try:
                self.collection.delete_collection("rag_docs")
                print("已清除舊的RAG資料庫")
            except:
                pass
        
        # AutoGen配置
        self.config_list = [
            {
                'base_url': self.base_url,
                'api_key': "fakekey",
                'model': self.llm_model,
            }
        ]
        
        self.llm_config = {
            "config_list": self.config_list,
            "temperature": 0.1,
        }
        
        self.setup_agents()
    
    def setup_agents(self):
        """設置AutoGen代理"""
        
        # 文檔篩選代理
        self.document_filter = autogen.AssistantAgent(
            name="DocumentFilter",
            llm_config=self.llm_config,
#             system_message="""您是文檔篩選專家。您的任務是：
# 1. 仔細閱讀提供的文檔內容
# 2. 判斷文檔是否與用戶問題相關
# 3. 如果相關，返回 "RELEVANT: [相關原因]"
# 4. 如果不相關，返回 "NOT_RELEVANT: [不相關原因]"
# 5. 請保持客觀和準確的判斷
# 請只返回判斷結果，不要添加其他內容。"""
            system_message="""您是東擎科技(ASRock Industrial)技術支援部門(TSD)的專業文檔篩選專家。
您的任務是精確判斷歷史技術支援文檔是否與用戶查詢相關。

**篩選標準（必須同時滿足）：**
1. **產品型號**：文檔中的產品型號必須與用戶詢問的型號一致，但有時候會有系列的問題可以以常理判斷
2. **型號識別規則**：
   - 4x4-7XXX = 7000系列
   - 4x4-6XXX = 6000系列  
   - 4x4-5XXX = 5000系列
   - 依此類推
3. **客戶**：如果用戶指定客戶，文檔中的客戶必須匹配
4. **問題類型相關**：文檔中描述的技術問題必須與用戶查詢的問題類型相關
5. 如果相關，返回 "RELEVANT: [相關原因]"
6. 如果不相關，返回 "NOT_RELEVANT: [不相關原因]"
7. 請保持客觀和準確的判斷
請只返回判斷結果，不要添加其他內容。"""

        )
        
        # 答案整合代理
        self.answer_synthesizer = autogen.AssistantAgent(
            name="AnswerSynthesizer",
            llm_config=self.llm_config,
            system_message="""您是答案整合專家。您的任務是：
1. 基於篩選後的相關文檔，為用戶問題提供綜合性答案
2. 整合所有相關信息，提供完整且準確的回答
3. 如果信息不足，請明確指出
4. 使用繁體中文回答
5. 回答完成後說 "TERMINATE"
請提供詳細、有用的答案。"""
        )
    
    def add_documents_from_directory(self, directory_path: str, file_patterns: List[str] = None):
        """從資料夾載入所有文檔到RAG資料庫"""
        if file_patterns is None:
            # 新增Word和Excel格式支持
            file_patterns = [
                "*.txt", "*.pdf", "*.md", "*.json",
                "*.docx", "*.doc",  # Word文檔
                "*.xlsx", "*.xls"   # Excel文檔
            ]
        
        print(f"開始載入資料夾: {directory_path}")
        print(f"支援的文件格式: {', '.join(file_patterns)}")
        
        if not os.path.exists(directory_path):
            print(f"錯誤: 資料夾 {directory_path} 不存在")
            return False
        
        total_files = 0
        successful_files = 0
        
        # 遍歷所有支援的文件類型
        for pattern in file_patterns:
            file_path_pattern = os.path.join(directory_path, pattern)
            files = glob.glob(file_path_pattern)
            
            for file_path in files:
                total_files += 1
                file_ext = os.path.splitext(file_path)[1].lower()
                
                try:
                    print(f"處理文件: {os.path.basename(file_path)}")
                    
                    # 根據文件類型載入內容
                    if file_ext == '.pdf':
                        success = self.add_document(file_path, doc_type="pdf")
                    elif file_ext in ['.txt', '.md']:
                        success = self.add_document(file_path, doc_type="txt")
                    elif file_ext == '.json':
                        success = self.add_json_document(file_path)
                    elif file_ext in ['.docx', '.doc']:
                        success = self.add_word_document(file_path)
                    elif file_ext in ['.xlsx', '.xls']:
                        success = self.add_excel_document(file_path)
                    else:
                        print(f"跳過不支援的文件類型: {file_path}")
                        continue
                    
                    if success:
                        successful_files += 1
                        
                except Exception as e:
                    print(f"處理文件 {file_path} 時發生錯誤: {e}")
        
        print(f"\n載入完成: 成功處理 {successful_files}/{total_files} 個文件")
        return successful_files > 0
    
    def add_word_document(self, file_path: str):
        """處理Word文檔 (.docx, .doc)"""
        try:
            print(f"正在處理Word文檔: {file_path}")
            
            # 讀取Word文檔
            doc = Document(file_path)
            
            # 提取所有段落文本
            full_text = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():  # 忽略空段落
                    full_text.append(paragraph.text.strip())
            
            # 提取表格內容
            for table in doc.tables:
                table_text = []
                for row in table.rows:
                    row_text = []
                    for cell in row.cells:
                        cell_text = cell.text.strip()
                        if cell_text:
                            row_text.append(cell_text)
                    if row_text:
                        table_text.append(" | ".join(row_text))
                
                if table_text:
                    full_text.append("表格內容:\n" + "\n".join(table_text))
            
            # 合併所有文本
            text = "\n\n".join(full_text)
            
            if not text.strip():
                print(f"警告: Word文檔 {file_path} 沒有文本內容")
                return False
            
            # 文本分割
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=850,
                chunk_overlap=100,  # Word文檔可能需要更多重疊
                length_function=len,
            )
            chunks = text_splitter.split_text(text)
            
            # 準備批量數據
            ids = []
            embeddings = []
            documents = []
            metadatas = []
            
            # 生成嵌入並準備數據
            for i, chunk in enumerate(chunks):
                response = ollama.embed(model=self.embedding_model, input=chunk)
                embedding = response["embeddings"][0]
                
                doc_id = f"{os.path.basename(file_path)}_{i}"
                ids.append(doc_id)
                embeddings.append(embedding)
                documents.append(chunk)
                metadatas.append({
                    "source": file_path, 
                    "chunk_id": i, 
                    "file_type": "docx",
                    "paragraphs_count": len(doc.paragraphs),
                    "tables_count": len(doc.tables)
                })
            
            # 批量添加到資料庫
            self.collection.add(
                ids=ids,
                embeddings=embeddings,
                documents=documents,
                metadatas=metadatas
            )
            
            print(f"成功添加Word文檔: {file_path}，共{len(chunks)}個片段")
            print(f"  - 段落數: {len(doc.paragraphs)}")
            print(f"  - 表格數: {len(doc.tables)}")
            return True
            
        except Exception as e:
            print(f"添加Word文檔失敗: {e}")
            return False
    
    def add_excel_document(self, file_path: str):
        """處理Excel文檔 (.xlsx, .xls)"""
        try:
            print(f"正在處理Excel文檔: {file_path}")
            
            # 讀取Excel文件
            try:
                # 讀取所有工作表
                excel_file = pd.ExcelFile(file_path)
                sheet_names = excel_file.sheet_names
                print(f"  - 發現 {len(sheet_names)} 個工作表: {sheet_names}")
                
                all_text = []
                
                for sheet_name in sheet_names:
                    print(f"  - 處理工作表: {sheet_name}")
                    
                    # 讀取工作表
                    df = pd.read_excel(file_path, sheet_name=sheet_name)
                    
                    if df.empty:
                        print(f"    工作表 {sheet_name} 為空，跳過")
                        continue
                    
                    # 構建工作表文本
                    sheet_text = [f"工作表: {sheet_name}"]
                    sheet_text.append(f"行數: {len(df)}, 列數: {len(df.columns)}")
                    sheet_text.append("列名: " + " | ".join(str(col) for col in df.columns))
                    sheet_text.append("")
                    
                    # 將DataFrame轉換為文本
                    # 限制每個工作表最多處理1000行，避免過大
                    max_rows = min(1000, len(df))
                    for i in range(max_rows):
                        row_data = []
                        for col in df.columns:
                            cell_value = df.iloc[i][col]
                            # 處理NaN值
                            if pd.isna(cell_value):
                                cell_value = ""
                            else:
                                cell_value = str(cell_value).strip()
                            row_data.append(cell_value)
                        
                        if any(row_data):  # 只添加非空行
                            sheet_text.append(" | ".join(row_data))
                    
                    # 如果有更多行，添加說明
                    if len(df) > max_rows:
                        sheet_text.append(f"... (還有 {len(df) - max_rows} 行數據)")
                    
                    all_text.append("\n".join(sheet_text))
                
                # 合併所有工作表文本
                text = "\n\n" + "="*50 + "\n\n".join(all_text)
                
            except Exception as e:
                print(f"讀取Excel文件時發生錯誤: {e}")
                return False
            
            if not text.strip():
                print(f"警告: Excel文檔 {file_path} 沒有數據內容")
                return False
            
            # 文本分割 - Excel文檔通常比較結構化，使用較大的chunk
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1200,  # Excel數據可以使用更大的chunk
                chunk_overlap=100,
                length_function=len,
            )
            chunks = text_splitter.split_text(text)
            
            # 準備批量數據
            ids = []
            embeddings = []
            documents = []
            metadatas = []
            
            # 生成嵌入並準備數據
            for i, chunk in enumerate(chunks):
                response = ollama.embed(model=self.embedding_model, input=chunk)
                embedding = response["embeddings"][0]
                
                doc_id = f"{os.path.basename(file_path)}_{i}"
                ids.append(doc_id)
                embeddings.append(embedding)
                documents.append(chunk)
                metadatas.append({
                    "source": file_path, 
                    "chunk_id": i, 
                    "file_type": "excel",
                    "sheets_count": len(sheet_names),
                    "sheet_names": sheet_names
                })
            
            # 批量添加到資料庫
            self.collection.add(
                ids=ids,
                embeddings=embeddings,
                documents=documents,
                metadatas=metadatas
            )
            
            print(f"成功添加Excel文檔: {file_path}，共{len(chunks)}個片段")
            print(f"  - 工作表數: {len(sheet_names)}")
            return True
            
        except Exception as e:
            print(f"添加Excel文檔失敗: {e}")
            return False
    
    def add_json_document(self, file_path: str):
        """專門處理JSON文檔"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            # 將JSON轉換為文本
            if isinstance(json_data, dict):
                text = json.dumps(json_data, ensure_ascii=False, indent=2)
            elif isinstance(json_data, list):
                text = "\n".join([json.dumps(item, ensure_ascii=False) for item in json_data])
            else:
                text = str(json_data)
            
            # 文本分割
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=850,
                chunk_overlap=0,
                length_function=len,
            )
            chunks = text_splitter.split_text(text)
            
            # 準備批量數據
            ids = []
            embeddings = []
            documents = []
            metadatas = []
            
            # 生成嵌入並準備數據
            for i, chunk in enumerate(chunks):
                response = ollama.embed(model=self.embedding_model, input=chunk)
                embedding = response["embeddings"][0]  # ollama返回的是列表的列表
                
                doc_id = f"{os.path.basename(file_path)}_{i}"
                ids.append(doc_id)
                embeddings.append(embedding)
                documents.append(chunk)
                metadatas.append({
                    "source": file_path, 
                    "chunk_id": i, 
                    "file_type": "json"
                })
            
            # 批量添加到資料庫
            self.collection.add(
                ids=ids,
                embeddings=embeddings,
                documents=documents,
                metadatas=metadatas
            )
            
            print(f"成功添加JSON文檔: {file_path}，共{len(chunks)}個片段")
            return True
            
        except Exception as e:
            print(f"添加JSON文檔失敗: {e}")
            return False
    
    def add_document(self, file_path: str, doc_type: str = "txt"):
        """添加單個文檔到RAG資料庫"""
        try:
            # 根據文件類型載入文檔
            if doc_type.lower() == "pdf":
                loader = PyPDFLoader(file_path)
                documents = loader.load()
                text = "\n".join([doc.page_content for doc in documents])
            else:
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
            
            # 文本分割
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=850,
                chunk_overlap=0,
                length_function=len,
            )
            chunks = text_splitter.split_text(text)
            
            # 準備批量數據
            ids = []
            embeddings = []
            documents = []
            metadatas = []
            
            # 生成嵌入並準備數據
            for i, chunk in enumerate(chunks):
                response = ollama.embed(model=self.embedding_model, input=chunk)
                embedding = response["embeddings"][0]  # ollama返回的是列表的列表
                
                doc_id = f"{os.path.basename(file_path)}_{i}"
                ids.append(doc_id)
                embeddings.append(embedding)
                documents.append(chunk)
                metadatas.append({
                    "source": file_path, 
                    "chunk_id": i, 
                    "file_type": doc_type
                })
            
            # 批量添加到資料庫
            self.collection.add(
                ids=ids,
                embeddings=embeddings,
                documents=documents,
                metadatas=metadatas
            )
            
            print(f"成功添加文檔: {file_path}，共{len(chunks)}個片段")
            return True
            
        except Exception as e:
            print(f"添加文檔失敗: {e}")
            return False
    
    def list_loaded_documents(self):
        """顯示已載入的文檔列表"""
        try:
            all_docs = self.collection.get(include=["metadatas"])
            
            # 統計不同的文件
            file_stats = {}
            for metadata in all_docs["metadatas"]:
                source = metadata.get("source", "unknown")
                filename = os.path.basename(source)
                file_type = metadata.get("file_type", "unknown")
                
                if filename not in file_stats:
                    file_stats[filename] = {
                        "chunks": 0,
                        "type": file_type,
                        "path": source,
                        "extra_info": {}
                    }
                file_stats[filename]["chunks"] += 1
                
                # 添加額外信息
                if file_type == "docx":
                    file_stats[filename]["extra_info"]["paragraphs"] = metadata.get("paragraphs_count", 0)
                    file_stats[filename]["extra_info"]["tables"] = metadata.get("tables_count", 0)
                elif file_type == "excel":
                    file_stats[filename]["extra_info"]["sheets"] = metadata.get("sheets_count", 0)
                    file_stats[filename]["extra_info"]["sheet_names"] = metadata.get("sheet_names", [])
            
            print(f"\n已載入的文檔列表 (共 {len(file_stats)} 個文件):")
            for filename, stats in file_stats.items():
                extra_info = ""
                if stats["type"] == "docx" and stats["extra_info"]:
                    extra_info = f" [段落:{stats['extra_info'].get('paragraphs', 0)}, 表格:{stats['extra_info'].get('tables', 0)}]"
                elif stats["type"] == "excel" and stats["extra_info"]:
                    sheets = stats['extra_info'].get('sheets', 0)
                    extra_info = f" [工作表:{sheets}]"
                
                # 選擇適當的圖標
                icon = {
                    "txt": "📄", "md": "📝", "pdf": "📕", 
                    "json": "📋", "docx": "📘", "excel": "📊"
                }.get(stats["type"], "📄")
                
                print(f"  {icon} {filename} ({stats['type']}) - {stats['chunks']} 個片段{extra_info}")
                
        except Exception as e:
            print(f"獲取文檔列表失敗: {e}")
    
    def search_documents(self, query: str, n_results: int = 4) -> List[Dict]:
        """搜索最相關的文檔"""
        try:
            # 生成查詢嵌入
            response = ollama.embed(model=self.embedding_model, input=query)
            query_embedding = [response["embeddings"][0]]  # 包裝成二維列表
            
            # 搜索相似文檔
            results = self.collection.query(
                query_embeddings=query_embedding,
                n_results=n_results
            )
            
            documents = []
            for i in range(len(results["documents"][0])):
                documents.append({
                    "content": results["documents"][0][i],
                    "metadata": results["metadatas"][0][i],
                    "distance": results["distances"][0][i]
                })
            
            return documents
            
        except Exception as e:
            print(f"文檔搜索失敗: {e}")
            return []
    
    def filter_documents(self, query: str, documents: List[Dict]) -> List[Dict]:
        """使用第一個LLM篩選文檔"""
        relevant_docs = []
        
        for doc in documents:
            # 為每個文檔篩選創建新的用戶代理
            filter_user_proxy = autogen.UserProxyAgent(
                name="filter_user_proxy",
                human_input_mode="NEVER",
                max_consecutive_auto_reply=1,
                is_termination_msg=lambda x: True,
                code_execution_config=False,
                llm_config=self.llm_config,
            )
            
            # 構造篩選提示
            filter_prompt = f"""
用戶問題: {query}

文檔內容:
{doc['content']}

請判斷此文檔是否與用戶問題相關。
"""
            
            # 與文檔篩選代理對話
            filter_user_proxy.initiate_chat(
                self.document_filter,
                message=filter_prompt,
                max_turns=1
            )
            
            try:
                last_message = filter_user_proxy.last_message(self.document_filter)
                if last_message and "NOT_RELEVANT:" in last_message["content"]:
                    continue
                else:
                    relevant_docs.append(doc)
            except Exception as e:
                print(f"獲取篩選結果失敗: {e}")
                # 如果篩選失敗，保留文檔
                relevant_docs.append(doc)
        
        return relevant_docs
    
    def generate_answer(self, query: str, relevant_docs: List[Dict]) -> str:
        """使用第二個LLM生成最終答案"""
        if len(relevant_docs) == 0:
            return "沒有找到相關文檔來回答您的問題。"
        
        # 為答案生成創建新的用戶代理
        synthesis_user_proxy = autogen.UserProxyAgent(
            name="synthesis_user_proxy",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
            code_execution_config=False,
            llm_config=self.llm_config,
        )
        
        # 構造整合提示
        context = "\n\n".join([f"文檔{i+1}:\n{doc['content']}" 
                              for i, doc in enumerate(relevant_docs)])
        
        synthesis_prompt = f"""
基於以下相關文檔回答用戶問題:

用戶問題: {query}

相關文檔:
{context}

請提供綜合性的答案:
"""
        
        # 與答案整合代理對話
        synthesis_user_proxy.initiate_chat(
            self.answer_synthesizer,
            message=synthesis_prompt,
            max_turns=1
        )
        
        # 獲取最終答案
        try:
            last_message = synthesis_user_proxy.last_message(self.answer_synthesizer)
            return last_message["content"] if last_message else "生成答案失敗"
        except Exception as e:
            print(f"獲取答案失敗: {e}")
            return "生成答案過程中發生錯誤"
    
    def query(self, question: str) -> str:
        """完整的RAG查詢流程"""
        print(f"\n開始處理問題: {question}")
        
        # 步驟1: 搜索相關文檔
        print("1. 搜索相關文檔...")
        documents = self.search_documents(question, n_results=4)
        print(f"找到 {len(documents)} 個候選文檔")
        
        if not documents:
            return "沒有找到相關文檔"
        
        # 顯示找到的文檔檔案名稱和類型
        print("找到的文檔:")
        for i, doc in enumerate(documents, 1):
            filename = os.path.basename(doc['metadata']['source'])
            file_type = doc['metadata'].get('file_type', 'unknown')
            distance = doc['distance']
            print(f"  {i}. {filename} ({file_type}) - 相似度: {1-distance:.3f}")
        
        # 步驟2: 篩選文檔
        print("\n2. 篩選相關文檔...")
        relevant_docs = self.filter_documents(question, documents)
        print(f"篩選後保留 {len(relevant_docs)} 個相關文檔")
        
        # 顯示篩選後保留的文檔
        if relevant_docs:
            print("保留的文檔:")
            for i, doc in enumerate(relevant_docs, 1):
                filename = os.path.basename(doc['metadata']['source'])
                file_type = doc['metadata'].get('file_type', 'unknown')
                print(f"  {i}. {filename} ({file_type})")
        
        # 步驟3: 生成答案
        print("\n3. 生成最終答案...")
        answer = self.generate_answer(question, relevant_docs)
        
        return answer


# 使用示例
def main():
    # 初始化自定義RAG系統
    rag_system = CustomRAGAgentSystem(
        reset_db=False, 
        db_path="./custom_json_rag_db"
    )
    
    # 從資料夾載入所有文檔（現在支援Word和Excel）
    # directory_path = "/home/asri/TSD_issue/test"  # 您的文檔資料夾路徑
    directory_path = "/home/asri/TSD_issue/tsd_issue_summary_1140410"
    
    # if os.path.exists(directory_path):
    #     print("=== 從資料夾載入文檔 ===")
    #     rag_system.add_documents_from_directory(
    #         directory_path=directory_path,
    #         file_patterns=[
    #             "*.txt", "*.pdf", "*.md", "*.json",
    #             "*.docx", "*.doc",   # Word文檔
    #             "*.xlsx", "*.xls"    # Excel文檔
    #         ]
    #     )
    
    # # 顯示已載入的文檔
    # rag_system.list_loaded_documents()
    
    # 互動式查詢
    print("\n=== 多格式RAG AI Agent 已就緒 ===")
    print("現在支援: TXT, PDF, MD, JSON, DOCX, DOC, XLSX, XLS")
    print("輸入 'quit' 退出程式")
    print("輸入 'list' 查看已載入的文檔")
    
    while True:
        question = input("\n請輸入您的問題: ")
        if question.lower() == 'quit':
            break
        elif question.lower() == 'list':
            rag_system.list_loaded_documents()
            continue
        
        answer = rag_system.query(question)
        print(f"\n答案: {answer}")

if __name__ == "__main__":
    main()



=== 多格式RAG AI Agent 已就緒 ===
現在支援: TXT, PDF, MD, JSON, DOCX, DOC, XLSX, XLS
輸入 'quit' 退出程式
輸入 'list' 查看已載入的文檔

開始處理問題: 找有關BE200發生過的issue
1. 搜索相關文檔...
找到 4 個候選文檔
找到的文檔:
  1. 11.txt (txt) - 相似度: 0.667
  2. 15.txt (txt) - 相似度: 0.601
  3. 494.txt (txt) - 相似度: 0.588
  4. 157.txt (txt) - 相似度: 0.587

2. 篩選相關文檔...
filter_user_proxy (to DocumentFilter):


用戶問題: 找有關BE200發生過的issue

文檔內容:
編號與日期: (11) 20241014
客戶: TW Jetone
購買產品: IMB-X1314
問題描述：
TW Jetone 公司反映其使用的 IMB-X1314 搭配 Intel BE200 在清除 CMOS 或 BIOS 預設載入後，裝置管理員無法偵測該裝置。進一步測試顯示，在 S3、S4、S5 狀態下無法正常重置 BE200，導致裝置無法被系統辨識。

解決辦法：
經技術支援部門（TSD）和研發部門（RD）確認，問題源於早期 ADL-S 主板的 PCHU1 接 +3V 造成 S3/S4/S5 時無法正常拉低 BUF_PLT_RST#，進而影響 BE200 的重置。建議的解決方案是將 PCHU1 改接至 +3VSB，並通過 AND 閘控制 M2/PCIe 插槽的 BUF_PLT_RST#。目前，建議客戶若需要 WiFi 7 功能，可以先更換 Qualcomm NCM865 模組作為替代方案。未來 PCB 改版時將一併導入此修正。

專有名詞解釋：
- IMB-X1314：東擎科技生產的一款工業級主板。
- Intel BE200：Intel 的一款整合 WiFi 7 和 Bluetooth 的 M.2 模組。
- BUF_PLT_RST#：一種信號線，用於控制系統重置。
- PCHU1：主板上的特定連接點，用於供電控制。
- +3VSB：待機電壓，即使系統處於待機模式也能提供電力。

請判斷此文檔是否與用戶問